In [167]:
item = {
  "purpose": "sql query interpretation",
  "system-prompt": "You are a SQL developer tasked with deriving insights from a database using SQL.\n- Don't create any tables or views or functions.\n- Use the provided schema to answer the question.\n- Don't include schema in response.",
  "steps": [
    {
      "name": "Outline Creation",
      "output": "<OUTLINE>",
      "instruction": "Craft an outline to answer: <QUESTION>.\nContext: <CONTEXT>.\nStart by giving the list of tables and the columns necessary to answer the question then provide logical steps to build the query, don't use any SQL.",
      "inputs": ["<QUESTION>", "<CONTEXT>"]
    },
    {
      "name": "Revise Outline",
      "system-prompt": "You are a helpful assistant. Analyse the outline provided and revise it if necessary. Spot any inconsistencies and correct them. Don't give the answer in SQL.",
      "output": "<REVISED_OUTLINE>",
      "instruction": "Analyse the following outline and correct it if necessary:\n<OUTLINE>\nQuestion: <QUESTION>.\nContext: \n<CONTEXT>.\nDon't use any SQL.",
      "inputs": ["<QUESTION>", "<CONTEXT>", "<OUTLINE>"]
    },
    {
      "name": "Generate SQL",
      "output": "<SQL_SNIPPET>",
      "instruction": "Using the outline, create a SQL query.\nOutline: <REVISED_OUTLINE>\nContext: <CONTEXT>\nQuestion to answer: <QUESTION>\nTarget Database: <DATABASE>",
      "inputs": ["<REVISED_OUTLINE>", "<CONTEXT>", "<DATABASE>", "<QUESTION>"]
    },
    {
      "name": "SQL Verification",
      "output": "<VERIFIED_SQL>",
      "instruction": "Verify the SQL query for correctness and relevance.\nEnsure all referenced tables and columns exist.\nQuestion: <QUESTION>\nContext: <CONTEXT>\nSQL Query: <SQL_SNIPPET>\nGive the answer in one 1 sql code snippet for <DATABASE> at the end.",
      "inputs": ["<SQL_SNIPPET>", "<QUESTION>", "<CONTEXT>", "<DATABASE>"]
    }
  ],
  "final-output": "<VERIFIED_SQL>"
}


In [168]:
# Open the text file in read mode
with open('./init_db.sql', 'r') as file:
    # Read the content of the file into a variable
    context = file.read()
question="Give me the total revenue made by provider."

inputs = {
    "<CONTEXT>": context,
    "<QUESTION>": question,
    "<DATABASE>": "Postgres"
}

In [169]:
from llm_interface import LLMClient

client = LLMClient("./config.yaml")

In [175]:
input_collection = inputs
for step in item['steps']:
    print(step['name'], "...")
    system_prompt = step["system-prompt"] if "system-prompt" in step.keys() else item['system-prompt']
    prompt = step["instruction"]
    for input_needed in step['inputs']:
        if input_needed not in input_collection.keys():
            raise AttributeError(f"{input_needed} input is not available.")
        print(f"Processing {input_needed}...")
        prompt = prompt.replace(input_needed, input_collection[input_needed])
    response = client.ask_question(prompt=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
          ])
    input_collection[step['output']] = response

Outline Creation ...
Processing <QUESTION>...
Processing <CONTEXT>...
Revise Outline ...
Processing <QUESTION>...
Processing <CONTEXT>...
Processing <OUTLINE>...
Generate SQL ...
Processing <REVISED_OUTLINE>...
Processing <CONTEXT>...
Processing <DATABASE>...
Processing <QUESTION>...
SQL Verification ...
Processing <SQL_SNIPPET>...
Processing <QUESTION>...
Processing <CONTEXT>...
Processing <DATABASE>...


In [176]:
print(input_collection["<OUTLINE>"])

 Tables and columns necessary:

1. Orders: OrderID, TotalAmount, CustomerID
2. OrderDetails: OrderID, ProductID, Quantity, UnitPrice
3. Payments: PaymentID, OrderID, Amount

Logical steps to build the query:

1. Start with the Orders table and join it with the OrderDetails table on the OrderID.
2. Join this result with the Products table using the ProductID from OrderDetails.
3. Join it further with the Payments table on the OrderID.
4. Group the final result by the provider (which can be derived from the SupplierID in the Products table).
5. Sum up the TotalAmount from Orders and Amount from Payments to get the total revenue.

Note: The above steps assume that each product is associated with only one supplier. If a product can be supplied by multiple suppliers, then we need to determine how to assign revenue to individual suppliers.


In [177]:
print(input_collection["<REVISED_OUTLINE>"])

 The outline provided is mostly correct, but there seems to be a mix-up between the "Suppliers" and "Providers." I'll assume you're looking for the total revenue by supplier. Here's the revised outline:

1. Start with the "Orders" table and join it with the "OrderDetails" table on the "OrderID".
2. Join this result with the "Products" table using the "ProductID" from "OrderDetails".
3. Join it further with the "Suppliers" table using the "SupplierID" from "Products".
4. Join it with the "Payments" table on the "OrderID".
5. Group the final result by the supplier (which can be found in the "Suppliers" table).
6. Sum up the "TotalAmount" from "Orders" and "Amount" from "Payments" to get the total revenue.

The context you provided contains the correct schema, and the tables and columns necessary for this query are:

- Orders: OrderID, TotalAmount, CustomerID
- OrderDetails: OrderID, ProductID, Quantity, UnitPrice
- Products: ProductID, SupplierID
- Suppliers: SupplierID, SupplierName
- P

In [178]:
print(input_collection["<SQL_SNIPPET>"])

 WITH order\_product\_supplier AS (
SELECT o.OrderID, o.TotalAmount, p.SupplierID
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
),
payments\_sum AS (
SELECT OrderID, SUM(Amount) AS PaymentSum
FROM Payments
GROUP BY OrderID
)
SELECT s.SupplierID, s.SupplierName, SUM(op.TotalAmount + coalesce(ps.PaymentSum, 0)) AS TotalRevenue
FROM order\_product\_supplier op
JOIN Suppliers s ON op.SupplierID = s.SupplierID
LEFT JOIN payments\_sum ps ON op.OrderID = ps.OrderID
GROUP BY s.SupplierID, s.SupplierName
ORDER BY TotalRevenue DESC;


In [179]:
print(input_collection["<VERIFIED_SQL>"])

 Your SQL query looks good and should provide the correct results. It first creates a CTE (Common Table Expression) named `order_product_supplier` to get the `OrderID`, `TotalAmount`, and `SupplierID` from the `Orders`, `OrderDetails`, and `Products` tables. Then, it creates another CTE named `payments_sum` to calculate the total payment amount for each order.

Finally, it joins these two CTEs with the `Suppliers` table to get the supplier information and calculate the total revenue for each supplier by summing the `TotalAmount` and the `PaymentSum`. The `coalesce` function is used to handle the case where a payment is not made for an order yet.

Here's the SQL code snippet for Postgres:
```vbnet
WITH order_product_supplier AS (
SELECT o.OrderID, o.TotalAmount, p.SupplierID
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
),
payments_sum AS (
SELECT OrderID, SUM(Amount) AS PaymentSum
FROM Payments
GROUP BY OrderID
)
SELECT s.Sup